In [ ]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
from functools import lru_cache
from jdt import Jdt

In [ ]:
VIDEO_FILE = './heavy/2021-04-25.mp4'

In [ ]:
class GetFrame:
    # to improve: https://stackoverflow.com/questions/33650974/opencv-python-read-specific-frame-using-videocapture
    def __init__(self):
        self.cache = []
        self.videoCapture = None
        self.newVideoCapture()
    
    def newVideoCapture(self):
        self.release()
        self.videoCapture = cv.VideoCapture(VIDEO_FILE)
        self.cursor = 0
    
    def next(self):
        ret, frame = self.videoCapture.read()
        self.cursor += 1
        assert ret
        return frame
    
    def release(self):
        if self.videoCapture is not None:
            self.videoCapture.release()
    
    @lru_cache()
    def __call__(self, frame_i):
        if frame_i < self.cursor:
            self.newVideoCapture()
        with Jdt(frame_i - self.cursor, UPP = 4) as j:
            while self.cursor < frame_i:
                j.acc()
                self.next()
        assert frame_i == self.cursor
        frame = self.next()
        swapped = frame.copy()
        swapped[:,:,0] = frame[:,:,2]
        swapped[:,:,2] = frame[:,:,0]
        return swapped

getFrame = GetFrame()

def widePlot(w = 16, h = 9):
    fig = plt.gcf()
    fig.set_size_inches(w, h)
def view(frame):
    plt.imshow(frame)
    widePlot()
    plt.show()

In [ ]:
def channel(frame, z_keep):
    result = frame.copy()
    for z in range(3):
        if z != z_keep:
            result[:, :, z] = frame[:, :, z_keep] * .5
    return result
def normalize(frame):
    ceil  = np.max(frame)
    floor = np.min(frame)
    if 0 <= floor < 1 and 254 < ceil <= 255:
        return frame
    return np.rint((frame - floor) / (ceil - floor) * 255)
def whiten(frame):
    # convert one-channel frame to three-channel
    w, h = frame.shape
    result = np.zeros((w, h, 3), dtype=np.int16)
    frame = normalize(frame)
    result[:, :, 0] = frame
    result[:, :, 1] = frame
    result[:, :, 2] = frame
    return result

In [ ]:
# sample = getFrame(1720)[450:950, :, :]
sample = getFrame(3050)[450:950, :, :]

## 分信道

In [ ]:
view(np.concatenate([sample] + [channel(sample, z) for z in range(3)]))

用蓝色非常合理。  
SM在绿色和红色下几乎不可见。  

## 麦片隐身术

remove cereal body

In [ ]:
view(whiten(sample[:, :, 2] - 2 * (sample[:, :, 1])))

linear system

In [ ]:
x, y = np.linalg.solve(np.array([[-5,11],[26,43]]), np.array([[-22],[-56]]))
x, y

In [ ]:
view(np.concatenate([
    sample, 
    whiten(.65 * sample[:, :, 0] - 1.7 * sample[:, :, 1] + sample[:, :, 2]), 
    channel(sample, 2),
]))

The above is fit from one pixel.  
next,  
## Fine tune

In [ ]:
view(np.concatenate([
    whiten((1.2 + 0) * sample[:, :, 0] - (1.9 + x) * sample[:, :, 1] + sample[:, :, 2]) 
    for x in np.linspace(-.5, .5, 5)
]))

In [ ]:
invisiblized = 255 - normalize((1.2 + 0) * sample[:, :, 0] - (1.9 + 0) * sample[:, :, 1] + sample[:, :, 2])
view(np.concatenate([
    255 - whiten(.65 * sample[:, :, 0] - 1.7 * sample[:, :, 1] + sample[:, :, 2]), 
    whiten(invisiblized), 
]))

Conclusion: big success.  
Notice! The light from the top of the box also automatically disappeared!  
This means the color of Slime Mold has very good features in terms of inductive biases.  

## Threshold? Low-pass? 

In [ ]:
view(np.concatenate([
    whiten(invisiblized), 
    *[whiten(normalize(np.clip(invisiblized - x, 0, 255))) for x in (
        150, 160, 170, 180, 190, 
    )], 
]))

low-pass

In [ ]:
N = 32
kernel = np.ones((N, N), np.float32) / N**2
blurred = cv.filter2D(invisiblized, -1, kernel)
view(np.concatenate([
    whiten(normalize(np.clip(invisiblized - 155, 0, 255))),
    whiten(normalize(np.clip(blurred      - 155, 0, 255))), 
]))

Too much sacrifice. No go! 

In [ ]:
darkened   = normalize(np.clip(invisiblized - 170, 0, 255))
overexpose = normalize(np.clip(darkened * 2, 0, 255))
view(np.concatenate([
    whiten(darkened),
    whiten(overexpose), 
    sample, 
    whiten(normalize(np.clip(50 - normalize(sample[:, :, 2]), 0, 255))), 
]))

初衷：实现人眼功能  
现状：超越人眼功能

## write video

In [ ]:
def processFrame(frame, y1, y2):
    sample = frame[y1:y2, :, :]
    invisiblized = 255 - normalize((1.2 + 0) * sample[:, :, 0] - (1.9 + 0) * sample[:, :, 1] + sample[:, :, 2])
    darkened   = normalize(np.clip(invisiblized - 170, 0, 255))
    return whiten(normalize(np.clip(darkened * 2, 0, 255)))
def writeVideo(y1 = 450, y2 = 1650):
    out = cv.VideoWriter('./heavy/temp.mp4', cv.VideoWriter_fourcc('M','J','P','G'), 10, (3264, y2 - y1))
    cap = cv.VideoCapture(VIDEO_FILE)
    try:
        i = 0
        while True:
            if i % 1 == 0:
                print(i, end='\r', flush = True)
            ret, frame = cap.read()
            if not ret:
                break
            swapped = frame.copy()
            swapped[:,:,0] = frame[:,:,2]
            swapped[:,:,2] = frame[:,:,0]
            p = processFrame(swapped, y1, y2)
            out.write(p)
            i += 1
    finally:
        cap.release()
        out.release()
writeVideo()

## diff

The problem with the previous method:  
Try changing "1720" to "3050"  

Why diff?  
Invisiblize the environment!  

In [ ]:
view(np.concatenate([
    getFrame(3300)[450:950, :, :] - 
    getFrame(3050)[450:950, :, :], 
    getFrame(3300)[450:950, :, :], 
    getFrame(3050)[450:950, :, :], 
]))